### Links
- PyTorch MNIST example - https://github.com/pytorch/examples/blob/master/mnist/main.py
- pytorch cifar10 code - https://github.com/kuangliu/pytorch-cifar/blob/master/main.py
- LeNet for CIFAR-10 - https://github.com/kuangliu/pytorch-cifar/blob/master/models/lenet.py

### My notes
- My Google Doc - https://docs.google.com/document/d/1nPvFULraomLmBp4IuNVe51YXV1DQsSzgQWtuaZhbkjo/edit
- Trello - https://trello.com/b/ycCGjD3A/mobile-dl
- Using webcam - https://docs.google.com/document/d/17BVx5yioNaV11mip4U9gRAO7wLIJf-aOpIITJPjqeks/edit#
- Installing pytorch - https://docs.google.com/document/d/1NfLbuhoOxonzS9FEeooyqpB90-62Gat-uZ2DOTaWRdg/edit#


In [1]:
import argparse
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
print(torch.__version__)
print(torch.__file__)

1.0.1.post2
/Users/william/.local/share/virtualenvs/william-oBc2a6gD/lib/python3.7/site-packages/torch/__init__.py


In [3]:
class Net(nn.Module):
    def __init__(self):
        """Create a LeNet-like neural net"""
        super(Net, self).__init__()
        # First conv is padded by 2 to work with MNIST image dimensions
        self.conv1 = nn.Conv2d(1, 6, 5, 1, padding=2)  # in_channels, out_channels, kernel_size, stride
        self.conv2 = nn.Conv2d(6, 16, 5, 1)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # in_features, out_features
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # torch.Size([64, 1, 28, 28])
        x = F.relu(self.conv1(x))  
        # torch.Size([64, 6, 28, 28])
        x = F.avg_pool2d(x, 2, 2)  # input, kernel_size, stride
        # torch.Size([64, 6, 14, 14])
        x = F.relu(self.conv2(x))
        # torch.Size([64, 16, 10, 10])
        x = F.avg_pool2d(x, 2, 2)
        # torch.Size([64, 16, 5, 5])
        # reshape for fully connected layers...
        x = x.view(-1, 16*5*5)  # what's the shape after this? 1 x 400?
        # torch.Size([64, 400])
        x = F.relu(self.fc1(x))
        # torch.Size([64, 120])
        x = F.relu(self.fc2(x))
        # torch.Size([64, 84])
        x = self.fc3(x)
        # torch.Size([64, 10])
        x = F.log_softmax(x, dim=1)
        # torch.Size([64, 10])
        return x


In [4]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                        help='input batch size for training (default: 64)')
parser.add_argument('--test-batch-size', type=int, default=1000, metavar='N',
                    help='input batch size for testing (default: 1000)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument('--lr', type=float, default=0.01, metavar='LR',
                    help='learning rate (default: 0.01)')
parser.add_argument('--momentum', type=float, default=0.5, metavar='M',
                    help='SGD momentum (default: 0.5)')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='disables CUDA training')
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')


_StoreAction(option_strings=['--log-interval'], dest='log_interval', nargs=None, const=None, default=10, type=<class 'int'>, choices=None, help='how many batches to wait before logging training status', metavar='N')

In [5]:
args = parser.parse_args(args=["--epochs=100"])
args

Namespace(batch_size=64, epochs=100, log_interval=10, lr=0.01, momentum=0.5, no_cuda=False, seed=1, test_batch_size=1000)

In [6]:
use_cuda = not args.no_cuda and torch.cuda.is_available()
use_cuda

False

In [7]:
torch.manual_seed(args.seed)

In [8]:
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cpu')

In [9]:
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
kwargs

{}

In [10]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)


In [11]:
model = Net().to(device)
model

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [12]:
optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.01
    momentum: 0.5
    nesterov: False
    weight_decay: 0
)

In [13]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)  # to do: needed?
        optimizer.zero_grad()  # what does this do? zero-out any previous gradient info?
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args.log_interval == 0:
            digits = int(math.log10(len(train_loader.dataset))) + 1
            print('Train Epoch: {} [{:05d}/{:05d} ({:02.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))


def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
            

In [14]:
for epoch in range(1, args.epochs + 1):
    train(args, model, device, train_loader, optimizer, epoch)
    test(args, model, device, test_loader)


Train Epoch: 1 [00000/60000 (00%)]	Loss: 2.307516
Train Epoch: 1 [00640/60000 (01%)]	Loss: 2.302698
Train Epoch: 1 [01280/60000 (02%)]	Loss: 2.306925
Train Epoch: 1 [01920/60000 (03%)]	Loss: 2.290555
Train Epoch: 1 [02560/60000 (04%)]	Loss: 2.293198
Train Epoch: 1 [03200/60000 (05%)]	Loss: 2.283789
Train Epoch: 1 [03840/60000 (06%)]	Loss: 2.281834
Train Epoch: 1 [04480/60000 (07%)]	Loss: 2.294910
Train Epoch: 1 [05120/60000 (09%)]	Loss: 2.288762
Train Epoch: 1 [05760/60000 (10%)]	Loss: 2.284253
Train Epoch: 1 [06400/60000 (11%)]	Loss: 2.284108
Train Epoch: 1 [07040/60000 (12%)]	Loss: 2.283559
Train Epoch: 1 [07680/60000 (13%)]	Loss: 2.267116
Train Epoch: 1 [08320/60000 (14%)]	Loss: 2.251754
Train Epoch: 1 [08960/60000 (15%)]	Loss: 2.268226
Train Epoch: 1 [09600/60000 (16%)]	Loss: 2.227675
Train Epoch: 1 [10240/60000 (17%)]	Loss: 2.209589
Train Epoch: 1 [10880/60000 (18%)]	Loss: 2.180263
Train Epoch: 1 [11520/60000 (19%)]	Loss: 2.187194
Train Epoch: 1 [12160/60000 (20%)]	Loss: 2.111264


Train Epoch: 2 [44160/60000 (74%)]	Loss: 0.095184
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.237732
Train Epoch: 2 [45440/60000 (76%)]	Loss: 0.130255
Train Epoch: 2 [46080/60000 (77%)]	Loss: 0.107744
Train Epoch: 2 [46720/60000 (78%)]	Loss: 0.161814
Train Epoch: 2 [47360/60000 (79%)]	Loss: 0.147443
Train Epoch: 2 [48000/60000 (80%)]	Loss: 0.335210
Train Epoch: 2 [48640/60000 (81%)]	Loss: 0.195709
Train Epoch: 2 [49280/60000 (82%)]	Loss: 0.197350
Train Epoch: 2 [49920/60000 (83%)]	Loss: 0.206623
Train Epoch: 2 [50560/60000 (84%)]	Loss: 0.086560
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.119120
Train Epoch: 2 [51840/60000 (86%)]	Loss: 0.247234
Train Epoch: 2 [52480/60000 (87%)]	Loss: 0.530244
Train Epoch: 2 [53120/60000 (88%)]	Loss: 0.139992
Train Epoch: 2 [53760/60000 (90%)]	Loss: 0.066490
Train Epoch: 2 [54400/60000 (91%)]	Loss: 0.147412
Train Epoch: 2 [55040/60000 (92%)]	Loss: 0.117429
Train Epoch: 2 [55680/60000 (93%)]	Loss: 0.167895
Train Epoch: 2 [56320/60000 (94%)]	Loss: 0.081822


Train Epoch: 4 [27520/60000 (46%)]	Loss: 0.063579
Train Epoch: 4 [28160/60000 (47%)]	Loss: 0.128139
Train Epoch: 4 [28800/60000 (48%)]	Loss: 0.050154
Train Epoch: 4 [29440/60000 (49%)]	Loss: 0.110234
Train Epoch: 4 [30080/60000 (50%)]	Loss: 0.093762
Train Epoch: 4 [30720/60000 (51%)]	Loss: 0.083698
Train Epoch: 4 [31360/60000 (52%)]	Loss: 0.234583
Train Epoch: 4 [32000/60000 (53%)]	Loss: 0.116438
Train Epoch: 4 [32640/60000 (54%)]	Loss: 0.082451
Train Epoch: 4 [33280/60000 (55%)]	Loss: 0.171975
Train Epoch: 4 [33920/60000 (57%)]	Loss: 0.107099
Train Epoch: 4 [34560/60000 (58%)]	Loss: 0.082182
Train Epoch: 4 [35200/60000 (59%)]	Loss: 0.112403
Train Epoch: 4 [35840/60000 (60%)]	Loss: 0.263242
Train Epoch: 4 [36480/60000 (61%)]	Loss: 0.093134
Train Epoch: 4 [37120/60000 (62%)]	Loss: 0.088393
Train Epoch: 4 [37760/60000 (63%)]	Loss: 0.171260
Train Epoch: 4 [38400/60000 (64%)]	Loss: 0.120603
Train Epoch: 4 [39040/60000 (65%)]	Loss: 0.075770
Train Epoch: 4 [39680/60000 (66%)]	Loss: 0.182563


Train Epoch: 6 [10880/60000 (18%)]	Loss: 0.179768
Train Epoch: 6 [11520/60000 (19%)]	Loss: 0.077235
Train Epoch: 6 [12160/60000 (20%)]	Loss: 0.093548
Train Epoch: 6 [12800/60000 (21%)]	Loss: 0.116388
Train Epoch: 6 [13440/60000 (22%)]	Loss: 0.075331
Train Epoch: 6 [14080/60000 (23%)]	Loss: 0.051522
Train Epoch: 6 [14720/60000 (25%)]	Loss: 0.168242
Train Epoch: 6 [15360/60000 (26%)]	Loss: 0.018150
Train Epoch: 6 [16000/60000 (27%)]	Loss: 0.170659
Train Epoch: 6 [16640/60000 (28%)]	Loss: 0.054319
Train Epoch: 6 [17280/60000 (29%)]	Loss: 0.022771
Train Epoch: 6 [17920/60000 (30%)]	Loss: 0.065420
Train Epoch: 6 [18560/60000 (31%)]	Loss: 0.041836
Train Epoch: 6 [19200/60000 (32%)]	Loss: 0.078152
Train Epoch: 6 [19840/60000 (33%)]	Loss: 0.011966
Train Epoch: 6 [20480/60000 (34%)]	Loss: 0.052227
Train Epoch: 6 [21120/60000 (35%)]	Loss: 0.063684
Train Epoch: 6 [21760/60000 (36%)]	Loss: 0.058725
Train Epoch: 6 [22400/60000 (37%)]	Loss: 0.071756
Train Epoch: 6 [23040/60000 (38%)]	Loss: 0.070761


Train Epoch: 7 [55040/60000 (92%)]	Loss: 0.043936
Train Epoch: 7 [55680/60000 (93%)]	Loss: 0.103220
Train Epoch: 7 [56320/60000 (94%)]	Loss: 0.043820
Train Epoch: 7 [56960/60000 (95%)]	Loss: 0.017833
Train Epoch: 7 [57600/60000 (96%)]	Loss: 0.020905
Train Epoch: 7 [58240/60000 (97%)]	Loss: 0.071072
Train Epoch: 7 [58880/60000 (98%)]	Loss: 0.045497
Train Epoch: 7 [59520/60000 (99%)]	Loss: 0.067642

Test set: Average loss: 0.0682, Accuracy: 9785/10000 (98%)

Train Epoch: 8 [00000/60000 (00%)]	Loss: 0.051626
Train Epoch: 8 [00640/60000 (01%)]	Loss: 0.059167
Train Epoch: 8 [01280/60000 (02%)]	Loss: 0.009632
Train Epoch: 8 [01920/60000 (03%)]	Loss: 0.038707
Train Epoch: 8 [02560/60000 (04%)]	Loss: 0.101028
Train Epoch: 8 [03200/60000 (05%)]	Loss: 0.013050
Train Epoch: 8 [03840/60000 (06%)]	Loss: 0.049930
Train Epoch: 8 [04480/60000 (07%)]	Loss: 0.084299
Train Epoch: 8 [05120/60000 (09%)]	Loss: 0.018694
Train Epoch: 8 [05760/60000 (10%)]	Loss: 0.025290
Train Epoch: 8 [06400/60000 (11%)]	Loss

...

Train Epoch: 92 [41600/60000 (69%)]	Loss: 0.000228
Train Epoch: 92 [42240/60000 (70%)]	Loss: 0.000202
Train Epoch: 92 [42880/60000 (71%)]	Loss: 0.000110
Train Epoch: 92 [43520/60000 (72%)]	Loss: 0.000065
Train Epoch: 92 [44160/60000 (74%)]	Loss: 0.000023
Train Epoch: 92 [44800/60000 (75%)]	Loss: 0.000108
Train Epoch: 92 [45440/60000 (76%)]	Loss: 0.000004
Train Epoch: 92 [46080/60000 (77%)]	Loss: 0.000062
Train Epoch: 92 [46720/60000 (78%)]	Loss: 0.000255
Train Epoch: 92 [47360/60000 (79%)]	Loss: 0.000194
Train Epoch: 92 [48000/60000 (80%)]	Loss: 0.000123
Train Epoch: 92 [48640/60000 (81%)]	Loss: 0.000229
Train Epoch: 92 [49280/60000 (82%)]	Loss: 0.000014
Train Epoch: 92 [49920/60000 (83%)]	Loss: 0.000114
Train Epoch: 92 [50560/60000 (84%)]	Loss: 0.000311
Train Epoch: 92 [51200/60000 (85%)]	Loss: 0.000005
Train Epoch: 92 [51840/60000 (86%)]	Loss: 0.000149
Train Epoch: 92 [52480/60000 (87%)]	Loss: 0.000143
Train Epoch: 92 [53120/60000 (88%)]	Loss: 0.000102
Train Epoch: 92 [53760/60000 (9

Train Epoch: 94 [23040/60000 (38%)]	Loss: 0.000146
Train Epoch: 94 [23680/60000 (39%)]	Loss: 0.001434
Train Epoch: 94 [24320/60000 (41%)]	Loss: 0.000008
Train Epoch: 94 [24960/60000 (42%)]	Loss: 0.000006
Train Epoch: 94 [25600/60000 (43%)]	Loss: 0.000017
Train Epoch: 94 [26240/60000 (44%)]	Loss: 0.002138
Train Epoch: 94 [26880/60000 (45%)]	Loss: 0.000048
Train Epoch: 94 [27520/60000 (46%)]	Loss: 0.000062
Train Epoch: 94 [28160/60000 (47%)]	Loss: 0.000033
Train Epoch: 94 [28800/60000 (48%)]	Loss: 0.000157
Train Epoch: 94 [29440/60000 (49%)]	Loss: 0.000041
Train Epoch: 94 [30080/60000 (50%)]	Loss: 0.000013
Train Epoch: 94 [30720/60000 (51%)]	Loss: 0.000001
Train Epoch: 94 [31360/60000 (52%)]	Loss: 0.000022
Train Epoch: 94 [32000/60000 (53%)]	Loss: 0.000034
Train Epoch: 94 [32640/60000 (54%)]	Loss: 0.000166
Train Epoch: 94 [33280/60000 (55%)]	Loss: 0.000013
Train Epoch: 94 [33920/60000 (57%)]	Loss: 0.000975
Train Epoch: 94 [34560/60000 (58%)]	Loss: 0.001168
Train Epoch: 94 [35200/60000 (5

Train Epoch: 96 [04480/60000 (07%)]	Loss: 0.000012
Train Epoch: 96 [05120/60000 (09%)]	Loss: 0.000053
Train Epoch: 96 [05760/60000 (10%)]	Loss: 0.000401
Train Epoch: 96 [06400/60000 (11%)]	Loss: 0.000228
Train Epoch: 96 [07040/60000 (12%)]	Loss: 0.000041
Train Epoch: 96 [07680/60000 (13%)]	Loss: 0.000028
Train Epoch: 96 [08320/60000 (14%)]	Loss: 0.000878
Train Epoch: 96 [08960/60000 (15%)]	Loss: 0.000034
Train Epoch: 96 [09600/60000 (16%)]	Loss: 0.000004
Train Epoch: 96 [10240/60000 (17%)]	Loss: 0.000059
Train Epoch: 96 [10880/60000 (18%)]	Loss: 0.000090
Train Epoch: 96 [11520/60000 (19%)]	Loss: 0.000072
Train Epoch: 96 [12160/60000 (20%)]	Loss: 0.000424
Train Epoch: 96 [12800/60000 (21%)]	Loss: 0.000003
Train Epoch: 96 [13440/60000 (22%)]	Loss: 0.000230
Train Epoch: 96 [14080/60000 (23%)]	Loss: 0.000077
Train Epoch: 96 [14720/60000 (25%)]	Loss: 0.000073
Train Epoch: 96 [15360/60000 (26%)]	Loss: 0.000001
Train Epoch: 96 [16000/60000 (27%)]	Loss: 0.000426
Train Epoch: 96 [16640/60000 (2

Train Epoch: 97 [46720/60000 (78%)]	Loss: 0.000001
Train Epoch: 97 [47360/60000 (79%)]	Loss: 0.000031
Train Epoch: 97 [48000/60000 (80%)]	Loss: 0.000022
Train Epoch: 97 [48640/60000 (81%)]	Loss: 0.001175
Train Epoch: 97 [49280/60000 (82%)]	Loss: 0.000037
Train Epoch: 97 [49920/60000 (83%)]	Loss: 0.000032
Train Epoch: 97 [50560/60000 (84%)]	Loss: 0.000002
Train Epoch: 97 [51200/60000 (85%)]	Loss: 0.000008
Train Epoch: 97 [51840/60000 (86%)]	Loss: 0.000014
Train Epoch: 97 [52480/60000 (87%)]	Loss: 0.002761
Train Epoch: 97 [53120/60000 (88%)]	Loss: 0.000057
Train Epoch: 97 [53760/60000 (90%)]	Loss: 0.000066
Train Epoch: 97 [54400/60000 (91%)]	Loss: 0.000016
Train Epoch: 97 [55040/60000 (92%)]	Loss: 0.000019
Train Epoch: 97 [55680/60000 (93%)]	Loss: 0.000026
Train Epoch: 97 [56320/60000 (94%)]	Loss: 0.000183
Train Epoch: 97 [56960/60000 (95%)]	Loss: 0.000009
Train Epoch: 97 [57600/60000 (96%)]	Loss: 0.000174
Train Epoch: 97 [58240/60000 (97%)]	Loss: 0.000036
Train Epoch: 97 [58880/60000 (9

Train Epoch: 99 [28160/60000 (47%)]	Loss: 0.000170
Train Epoch: 99 [28800/60000 (48%)]	Loss: 0.000071
Train Epoch: 99 [29440/60000 (49%)]	Loss: 0.000050
Train Epoch: 99 [30080/60000 (50%)]	Loss: 0.000035
Train Epoch: 99 [30720/60000 (51%)]	Loss: 0.000045
Train Epoch: 99 [31360/60000 (52%)]	Loss: 0.000041
Train Epoch: 99 [32000/60000 (53%)]	Loss: 0.000088
Train Epoch: 99 [32640/60000 (54%)]	Loss: 0.001484
Train Epoch: 99 [33280/60000 (55%)]	Loss: 0.000665
Train Epoch: 99 [33920/60000 (57%)]	Loss: 0.000360
Train Epoch: 99 [34560/60000 (58%)]	Loss: 0.000040
Train Epoch: 99 [35200/60000 (59%)]	Loss: 0.000008
Train Epoch: 99 [35840/60000 (60%)]	Loss: 0.000035
Train Epoch: 99 [36480/60000 (61%)]	Loss: 0.000067
Train Epoch: 99 [37120/60000 (62%)]	Loss: 0.000013
Train Epoch: 99 [37760/60000 (63%)]	Loss: 0.000002
Train Epoch: 99 [38400/60000 (64%)]	Loss: 0.000006
Train Epoch: 99 [39040/60000 (65%)]	Loss: 0.000092
Train Epoch: 99 [39680/60000 (66%)]	Loss: 0.000406
Train Epoch: 99 [40320/60000 (6

In [15]:
torch.optim.Optimizer

torch.optim.optimizer.Optimizer

In [16]:
type(optimizer)

torch.optim.sgd.SGD

In [17]:
isinstance(optimizer, torch.optim.SGD)

True

In [18]:
issubclass(type(optimizer), torch.optim.Optimizer)

True